In [ ]:
import requests  # Import requests module for making HTTP requests
import os        # Import os module for interacting with the operating system
import time      # Import time module for adding delays in the script
from bs4 import BeautifulSoup  # Import BeautifulSoup for parsing HTML and XML documents
import pandas as pd  # Import pandas for data manipulation and analysis
import glob  # Import glob for file pattern matching and retrieval

In [ ]:
# Insert absolute paths

data_base_path = '[set_the_path]'
path_to_save = '[set_the_path]'

In [ ]:
def scr(name):
    '''Take the name of a .js file and return a df with scraped data'''
    # Open and read the .js file
    with open(name, 'r') as f:
        data = ' '.join([l for l in f.readlines()])

        # Extract relevant data sections
        start = data.find('<article')
        end = data.rfind('<\/button>");') + 10
        data = data[start: end]
        data = data.replace(r'\n', '\n')
        data = data.replace('\\', '')

    # Parse HTML using BeautifulSoup to extract news links
    soup = BeautifulSoup(data)
    lks = [a['href'] for a in soup.find_all('a', href=True)]  
    date = []
    title = []
    text = []

    # Iterate through each news link and extract date, title, and text
    for link in lks:
        try:
            page = requests.get(link)
            soup = BeautifulSoup(page.text)
            try:
                ttl = soup.find("title").text
            except:
                ttl = 'No title'
            try:
                txt = ' '.join([i.text for i in soup.find_all("p")]) 
            except:
                txt = 'No text'
            time.sleep(5)  # Pause for 5 seconds to avoid overloading the server
        except:
            ttl = 'No title'
            txt = 'No text'
        
        # Extract date from the news link
        dt = link.strip('https://www.1tv.ru/news/').split('/')[0]
        date.append(dt)
        title.append(ttl)
        text.append(txt)
    
    # Create a DataFrame with extracted data
    df = pd.DataFrame(columns=['link'])
    df['date'] = date
    df['link'] = lks
    df['title'] = title
    df['text'] = text
    return df

# List all .js documents in the folder using os.walk and glob
data_base_path = './data/'  # Directory containing .js files
jss = []
for root, dirs, files in os.walk(data_base_path):
    jss += glob.glob(os.path.join(root, '*.js'))

# Create an empty DataFrame
df1 = pd.DataFrame(columns=['link', 'date', 'title', 'text'])

# Apply the scraping function to each .js file and concatenate results
for i in jss:
    df = scr(i)
    df1 = pd.concat([df1, df], axis=0)

# Save the DataFrame as a pickle file
df1.to_pickle('path_to_save.pkl')